In [116]:
import pandas as pd
import numpy as np

In [117]:
df = pd.read_csv('similarity_scores.csv')
df.head()

,user_id,message_ts,action_ts,recipe_a,recipe_b,score
0,10141,28/07/2016 15:22,28/07/2016 16:32,548,292,2
1,10163,11/08/2016 12:06,11/08/2016 12:08,584,288,1
2,10163,15/08/2016 11:04,15/08/2016 11:11,288,553,3
3,10011,20/08/2016 22:42,20/08/2016 22:43,271,498,2
4,10141,25/08/2016 10:38,25/08/2016 11:27,585,89,1


In [118]:
recipes_df = pd.read_csv('recipes_info.csv')
recipes_df.head()

,recipe_id,country,country_secondary,dish_category,dish_type,diet_type,carbohydrate_base,carbohydrate_category,protein,protein_cut,protein_type,family_friendly,spice_level,prep_time
0,2,india,india,stove top / bowl food,curry,fish,basmati,rice,shellfish,prawns,fish & seafood,no,no spice,25
1,5,italy,italy,protein&veg,meat & side veg,meat,white potatoes,potato,beans,butter beans,pulses,no,mild,35
2,9,thailand,thailand,stove top / bowl food,curry,vegan,basmati,rice,tofu,protein not found,vegetarian,no,spicy,25
3,16,united kingdom,united kingdom,protein&veg,fish & side veg,fish,NaN,NaN,oily fish,trout,fish & seafood,no,no spice,20
4,19,tunisia,tunisia,protein&veg,egg dish,vegetarian,millet,grains,eggs,eggs,dairy & eggs,no,no spice,30


In [96]:
df.groupby('user_id').count()['message_ts']

user_id
10001     139
10011     476
10123     341
10127     210
10141    1835
10145     269
10163    1068
Name: message_ts, dtype: int64

In [42]:
df.groupby('user_id').mean()['score']

user_id
10001    1.446043
10011    1.686975
10123    1.791789
10127    1.528571
10141    1.557493
10145    1.572491
10163    1.575843
Name: score, dtype: float64

In [37]:
a = df['recipe_a'].drop_duplicates().tolist()
b = df['recipe_b'].drop_duplicates().tolist()

items = []

for i in a:
    items.append(i)
    for j in b:
        items.append(j)
        
len(set(items))

130

In [38]:
unique_scored = pd.DataFrame(set(items), columns=['id'])
j = recipes_df.merge(unique_scored, left_on='recipe_id', right_on='id', how='left')
print(j['id'].isnull().sum(), j['id'].notnull().sum())

132 126


In [39]:
j = recipes_df.merge(unique_scored, left_on='recipe_id', right_on='id', how='right')
print(j['id'].isnull().sum(), j['id'].notnull().sum())

0 130


In [40]:
j[j['recipe_id'].isnull()]['recipe_id']

126   NaN
127   NaN
128   NaN
129   NaN
Name: recipe_id, dtype: float64

In [41]:
unique_scored[unique_scored['id'] == 126]

,id


In [45]:
mean_rating_matrix = df.pivot_table(values='score', 
                                    index='recipe_a', 
                                    columns='recipe_b', 
                                    aggfunc='mean').fillna(0)

In [138]:
import scipy.sparse as sparse
import networkx as nx

edges = df.groupby(['recipe_a','recipe_b']).agg('mean').reset_index()[['recipe_a','recipe_b', 'score']]

graph = nx.from_pandas_dataframe(edges,'recipe_a','recipe_b','score')
adj_matrix =  nx.to_numpy_matrix(graph, weight='score')
nodes = [int(n) for n in graph.nodes()]
adj_matrix_df = pd.DataFrame(adj_matrix, index=nodes, columns=nodes)
adj_matrix_df.head()


,256,257,2,259,260,5,518,264,9,522,...,509,497,498,501,425,504,251,252,253,255
256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(adj_matrix_df, adj_matrix_df)
cos_sim_df = pd.DataFrame(cos_sim, index=adj_matrix_df.index, columns=adj_matrix_df.index)
cos_sim_df.head()

,256,257,2,259,260,5,518,264,9,522,...,509,497,498,501,425,504,251,252,253,255
256,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
257,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.707107,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
259,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.57735,0.0
260,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0


In [156]:
cos_sim_df[57].sort_values(ascending=False).head()

57     1.000000
288    0.901996
58     0.895597
498    0.894187
561    0.888435
Name: 57, dtype: float64

In [158]:
recipes_df[recipes_df['recipe_id'] == 57].T

,9
recipe_id,57
country,japan
country_secondary,japan
dish_category,soups
dish_type,ramen
diet_type,meat
carbohydrate_base,wholewheat noodle nests
carbohydrate_category,noodles
protein,chicken
protein_cut,skin off thigh chicken


In [160]:
recipes_df[recipes_df['recipe_id'] == 58].T

,10
recipe_id,58
country,japan
country_secondary,japan
dish_category,soups
dish_type,ramen
diet_type,vegan
carbohydrate_base,wholewheat noodle nests
carbohydrate_category,noodles
protein,tofu
protein_cut,NaN


In [163]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
scale = sc.fit_transform(adj_matrix_df)
item_rating_corr = pd.DataFrame(scale,
                                index=adj_matrix_df.index,
                                columns=adj_matrix_df.index)\
                    .corr(method='pearson')
item_rating_corr[57].sort_values(ascending=False).head()

57     1.000000
288    0.873375
58     0.864912
498    0.862874
561    0.855375
Name: 57, dtype: float64

In [150]:
a = adj_matrix_df.values
b = adj_matrix_df.values

In [152]:
pd.DataFrame(np.dot(a, b)/(np.sqrt(np.sum(a**2))*np.sqrt(np.sum(b**2))))

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,128,129
0,0.001097,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000219,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000219,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000439,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.001975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001975,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000878,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.004607,0.000000,0.000878,0.000000,0.000000,...,0.000000,0.000000,0.001251,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000219,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000878,0.000000,0.030924,0.000000,0.000000,...,0.000000,0.000000,0.026206,0.000274,0.000000,0.000000,0.000293,0.000439,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001097,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000219,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [81]:
def cosine_similarity(matrix_df):
    # utilise numpys matrix operations for fast computation
    a = matrix_df.values.copy()
    b = matrix_df.values.copy()

    all_recipes_by_n_recipes = np.repeat(a[np.newaxis, :, :], 
                                         a.shape[0],
                                         axis=0)

    all_recipes = b.reshape(b.shape[0], 
                            1, 
                            b.shape[1])

    a_dot_b = np.dot(a, b)
    print(a_dot_b)
#     union = np.sum(all_recipes_by_n_recipes != all_recipes, axis=2)*2 + intersect
#     jaccard_sim = intersect/union 

#     jaccard_sim_df = pd.DataFrame(jaccard_sim, index=base.index, columns=base.index)
    
#     return jaccard_sim_df


cosine_similarity(mean_rating_matrix)

ValueError: shapes (87,99) and (87,99) not aligned: 99 (dim 1) != 87 (dim 0)

In [58]:
item_rating_corr[2].sort_values(ascending=False).head()

recipe_b
2      1.000000
467   -0.011628
360   -0.011628
364   -0.011628
378   -0.011628
Name: 2, dtype: float64

In [59]:
recipes_df[recipes_df['recipe_id'] == 2].T

,0
recipe_id,2
country,india
country_secondary,india
dish_category,stove top / bowl food
dish_type,curry
diet_type,fish
carbohydrate_base,basmati
carbohydrate_category,rice
protein,shellfish
protein_cut,prawns


In [55]:
recipes_df[recipes_df['recipe_id'] == 288].T

,42
recipe_id,288
country,china
country_secondary,china
dish_category,protein&veg
dish_type,fish & side veg
diet_type,fish
carbohydrate_base,brown rice
carbohydrate_category,rice
protein,whitefish
protein_cut,basa
